# ResNet 50 como backbone
Se evalúa el desempeño del uso del modelo mask_rcnn_R_50_FPN_1x como backbone para el entrenamiento con el dataset TACO

Modelo adaptado de https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5

Se instalan Detectron2 y sus dependencias

In [ ]:
# Se instala constructor PyYAML para python
!pip install pyyaml==5.1

# Se instala versión 1.8 de pytorch por compatibilidad por detectron2 (Fuente: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Se instala librería detectron2 compatible con pytorch 1.8
# Fuente: https://detectron2.readthedocs.io/tutorials/install.html 
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
exit(0)  # Después de la instalación, se debe reiniciar el entorno de ejecución (Opción "restart runtime") 

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html


In [ ]:
# Se instala librería para monitoreo del tiempo de ejecución
!pip install ipython-autotime
%load_ext autotime

time: 1.67 ms (started: 2021-07-01 16:12:53 +00:00)


In [ ]:
# Se verifica el GPU asignado en Colab
!nvidia-smi

Thu Jul  1 16:12:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Se carga librerías de pytorch
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.8")

1.8.0+cu101 True
time: 494 ms (started: 2021-07-01 16:13:01 +00:00)


In [ ]:
# Se cargan librerías básicas
# detectron2 
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

time: 635 ms (started: 2021-07-01 16:13:05 +00:00)


Agregamos el dataset TACO

In [ ]:
# Se descarga conjunto de imágenes de TACO 
!gdown --id 1CT6ws8MLyqkhMRmafMl6wuhqZ1lgFuLg #https://drive.google.com/file/d/1CT6ws8MLyqkhMRmafMl6wuhqZ1lgFuLg/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1CT6ws8MLyqkhMRmafMl6wuhqZ1lgFuLg
To: /content/TACO.zip
2.62GB [00:16, 158MB/s] 
time: 51.4 s (started: 2021-07-01 16:13:09 +00:00)


In [ ]:
# Se descomprimen las imágenes 
!unzip TACO.zip

Archive:  TACO.zip
   creating: data/
  inflating: data/all_image_urls.csv  
  inflating: data/annotations.json   
  inflating: data/annotations_unofficial.json  
   creating: data/batch_1/
   creating: data/batch_10/
  inflating: data/batch_10/000000.jpg  
  inflating: data/batch_10/000001.jpg  
  inflating: data/batch_10/000002.jpg  
  inflating: data/batch_10/000003.jpg  
  inflating: data/batch_10/000004.jpg  
  inflating: data/batch_10/000005.jpg  
  inflating: data/batch_10/000006.jpg  
  inflating: data/batch_10/000007.jpg  
  inflating: data/batch_10/000008.jpg  
  inflating: data/batch_10/000009.jpg  
  inflating: data/batch_10/000010.jpg  
  inflating: data/batch_10/000011.jpg  
  inflating: data/batch_10/000012.jpg  
  inflating: data/batch_10/000013.jpg  
  inflating: data/batch_10/000014.jpg  
  inflating: data/batch_10/000015.jpg  
  inflating: data/batch_10/000016.jpg  
  inflating: data/batch_10/000017.jpg  
  inflating: data/batch_10/000018.jpg  
  inflating: data/batc

In [ ]:
# Se descarga las liberías para generación del conjunto de entrenamiento, de pruebas y validación
!gdown --id 1SnfUBgpoo6N_k5bgjSW3PwmKV0ElXqVD #https://drive.google.com/file/d/1SnfUBgpoo6N_k5bgjSW3PwmKV0ElXqVD/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1SnfUBgpoo6N_k5bgjSW3PwmKV0ElXqVD
To: /content/split_dataset.py
100% 3.35k/3.35k [00:00<00:00, 6.30MB/s]
time: 1.62 s (started: 2021-07-01 16:19:33 +00:00)


In [ ]:
# Se realiza la generación del conjunto de entrenamiento, de pruebas y validación
!python split_dataset.py --dataset_dir data --test_percentage 20 --val_percentage 0 --nr_trials 1

time: 521 ms (started: 2021-07-01 16:19:38 +00:00)


## Segmentación por instancias en TACO-1
Agrupando las imágenes en una sola clase denominada "Litter" la red detecta cualquier tipo de basura y la segmenta

In [ ]:
from detectron2.structures import BoxMode

def get_TACO_1_dicts(im_dir, dataset):
    json_file = os.path.join(im_dir, dataset)
    with open(json_file) as f:
        data = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(data['images']):
        record = {}
        
        filename = os.path.join(im_dir, v["file_name"])
        
        record["file_name"] = filename
        record["image_id"] = v["id"]
        record["height"] = v["height"]
        record["width"] = v["width"]
      
        objs = []
        for anno in data["annotations"]:
            if anno["image_id"] == v["id"]:
                obj = {
                    "bbox": anno["bbox"],
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "segmentation": anno["segmentation"],
                    "category_id": 0,
                }
                objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

time: 15.4 ms (started: 2021-07-01 16:19:43 +00:00)


In [ ]:
for d in ["train", "test"]:
    DatasetCatalog.register("TACO_1_" + d, lambda d=d: get_TACO_1_dicts("data","annotations_0_" + d + ".json"))
    MetadataCatalog.get("TACO_1_" + d).set(thing_classes=["Litter"])
TACO_1_metadata = MetadataCatalog.get("TACO_1_train")

time: 2.43 ms (started: 2021-07-01 16:19:49 +00:00)


## Entrenamiento del modelo con TACO_1

In [ ]:
# Generación de imágenes de pruebas
dataset_dicts = get_TACO_1_dicts("data","annotations_0_train.json")
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=TACO_1_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# Entrenamiento del modelo
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("TACO_1_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")  # Carga de los pesos del modelo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001  # valor de learning rate
cfg.SOLVER.MAX_ITER = 2000   # número de iteraciones
cfg.SOLVER.STEPS = []  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # se considera 1 ya que solo se detecta y segmenta la clase "Litter". (Fuente: https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[07/01 16:20:30 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
model_final_a54504.pkl: 178MB [00:02, 59.5MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.b

[07/01 16:20:37 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[07/01 16:21:03 d2.utils.events]:  eta: 0:38:38  iter: 19  total_loss: 1.911  loss_cls: 0.694  loss_box_reg: 0.1799  loss_mask: 0.6884  loss_rpn_cls: 0.2614  loss_rpn_loc: 0.08083  time: 1.2011  data_time: 0.5343  lr: 1.9981e-05  max_mem: 4991M
[07/01 16:21:30 d2.utils.events]:  eta: 0:42:10  iter: 39  total_loss: 1.56  loss_cls: 0.4597  loss_box_reg: 0.1563  loss_mask: 0.6551  loss_rpn_cls: 0.1724  loss_rpn_loc: 0.03509  time: 1.2682  data_time: 0.5854  lr: 3.9961e-05  max_mem: 4991M
[07/01 16:21:57 d2.utils.events]:  eta: 0:42:45  iter: 59  total_loss: 1.303  loss_cls: 0.2774  loss_box_reg: 0.1581  loss_mask: 0.5863  loss_rpn_cls: 0.1827  loss_rpn_loc: 0.06978  time: 1.2897  data_time: 0.5687  lr: 5.9941e-05  max_mem: 4991M
[07/01 16:22:23 d2.utils.events]:  eta: 0:40:41  iter: 79  total_loss: 0.9234  loss_cls: 0.1546  loss_box_reg: 0.1066  loss_mask: 0.5234  loss_rpn_cls: 0.06395  loss_rpn_loc: 0.02688  time: 1.2964  data_time: 0.5873  lr: 7.9921e-05  max_mem: 4991M
[07/01 16:22:48 

In [ ]:
# Métricas obtenidas del modelo usando el utilitario tensorboard
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
# Configuración del modelo para realizar predicciones sobre los datos de prueba
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # ruta del modelo ya entrenado
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # configuración del límite de detección
predictor = DefaultPredictor(cfg)

time: 996 ms (started: 2021-06-30 03:47:24 +00:00)


In [ ]:
# Función que lee los archivos de pruebas
def get_TACO_filenames(img_dir):
  json_file = os.path.join(img_dir, "annotations_0_test.json")
  with open(json_file) as f:
    data = json.load(f)
  filenames = []
  for image in data['images']:
    filenames.append(os.path.join(img_dir,image['file_name']))
  return filenames

time: 4.5 ms (started: 2021-06-30 03:47:25 +00:00)


In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_TACO_filenames("data")
for d in random.sample(dataset_dicts, 1):    
    im = cv2.imread(d)
    outputs = predictor(im)  
    v = Visualizer(im[:, :, ::-1],
                   metadata=TACO_1_metadata, 
                   scale=0.5,  
                   instance_mode=ColorMode.IMAGE_BW   # remueve los colores de los pixeles no segmentados. Solo disponible para modelos de segmentados
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# Predicción y evaluación del modelo con el conjunto de datos TACO_1.
# Se generará las métricas del bounding box y segmentación
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("TACO_1_test", ("bbox","segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "TACO_1_test")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

[06/30 04:02:49 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[06/30 04:02:49 d2.data.common]: Serializing 300 elements to byte tensors and concatenating them all ...
[06/30 04:02:49 d2.data.common]: Serialized dataset takes 0.42 MiB
[06/30 04:02:49 d2.evaluation.evaluator]: Start inference on 300 images


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[06/30 04:03:05 d2.evaluation.evaluator]: Inference done 11/300. 0.1112 s / img. ETA=0:06:38
[06/30 04:03:11 d2.evaluation.evaluator]: Inference done 16/300. 0.1050 s / img. ETA=0:05:59
[06/30 04:03:16 d2.evaluation.evaluator]: Inference done 23/300. 0.0993 s / img. ETA=0:04:55
[06/30 04:03:22 d2.evaluation.evaluator]: Inference done 28/300. 0.0996 s / img. ETA=0:04:51
[06/30 04:03:27 d2.evaluation.evaluator]: Inference done 33/300. 0.0982 s / img. ETA=0:04:43
[06/30 04:03:36 d2.evaluation.evaluator]: Inference done 34/300. 0.1057 s / img. ETA=0:05:57
[06/30 04:03:42 d2.evaluation.evaluator]: Inference done 38/300. 0.1061 s / img. ETA=0:05:53
[06/30 04:03:50 d2.evaluation.evaluator]: Inference done 41/300. 0.1089 s / img. ETA=0:06:18
[06/30 04:03:57 d2.evaluation.evaluator]: Inference done 47/300. 0.1085 s / img. ETA=0:06:02
[06/30 04:04:03 d2.evaluation.evaluator]: Inference done 51/300. 0.1074 s / img. ETA=0:05:54
[06/30 04:04:08 d2.evaluation.evaluator]: Inference done 55/300. 0.106

## Segmentación por instancias en TACO-10
Se agrupan las imágenes en 10 clases haciendo que la red identifique y segmente la basura para luego clasificarla en una de las categorías.

In [ ]:
!gdown --id 1ATP41SZ52M736-UzxQ6X_g5PbVfSHsNu #https://drive.google.com/file/d/1ATP41SZ52M736-UzxQ6X_g5PbVfSHsNu/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1ATP41SZ52M736-UzxQ6X_g5PbVfSHsNu
To: /content/map_10.csv
100% 1.47k/1.47k [00:00<00:00, 1.29MB/s]
time: 907 ms (started: 2021-06-30 04:07:58 +00:00)


In [ ]:
from detectron2.structures import BoxMode
import csv

with open('map_10.csv') as csvfile:
    spamreader = csv.reader(csvfile) 
    
    class10 = {} #transforma cualquiera de las 60 clases en 
    classes_10 = [] #contiene el nombre de las 10 clases
    for row in spamreader:
        if row[1] not in classes_10:
            classes_10.append(row[1])
    class_id = {} #devuelve el id de las 10 clases
    for idx, c in enumerate(classes_10):
        class_id[c] = idx
    csvfile.seek(0)
    for row in spamreader:
        class10[row[0]] = class_id[row[1]]

time: 6.31 ms (started: 2021-06-30 04:07:59 +00:00)


In [ ]:
def get_TACO_10_dicts(im_dir, dataset):
    json_file = os.path.join(im_dir, dataset)
    with open(json_file) as f:
        data = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(data['images']):
        record = {}
        
        filename = os.path.join(im_dir, v["file_name"])
        
        record["file_name"] = filename
        record["image_id"] = v["id"]
        record["height"] = v["height"]
        record["width"] = v["width"]
      
        objs = []
        for anno in data["annotations"]:
            if anno["image_id"] == v["id"]:
                obj = {
                    "bbox": anno["bbox"],
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "segmentation": anno["segmentation"],
                    "category_id": class10[data["categories"][anno["category_id"]]["name"]],
                }
                objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

time: 20.2 ms (started: 2021-06-30 04:07:59 +00:00)


In [ ]:
for d in ["train", "test"]:
    DatasetCatalog.register("TACO_10_" + d, lambda d=d: get_TACO_10_dicts("data","annotations_0_" + d + ".json"))
    MetadataCatalog.get("TACO_10_" + d).set(thing_classes=classes_10)

time: 2.37 ms (started: 2021-06-30 04:07:59 +00:00)


## Entrenamiento del modelo con TACO_1

In [ ]:
# Generación de imágenes de pruebas
TACO_10_metadata = MetadataCatalog.get("TACO_10_train")
dataset_dicts = get_TACO_10_dicts("data","annotations_0_train.json")
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=TACO_10_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# Predicción y evaluación del modelo con el conjunto de datos TACO_10.
# Se generará las métricas del bounding box y segmentación
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("TACO_10_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")  # carga de los pesos del modelo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001   # valor del learning rate
cfg.SOLVER.MAX_ITER = 2000    # número de iteraciones
cfg.SOLVER.STEPS = []  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10  # se considera 1 ya que solo se detecta y segmenta la clase "Litter". (Fuente https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[06/30 04:08:02 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in 

[06/30 04:08:06 d2.engine.train_loop]: Starting training from iteration 0
[06/30 04:08:34 d2.utils.events]:  eta: 0:11:18  iter: 19  total_loss: 3.571  loss_cls: 2.591  loss_box_reg: 0.1592  loss_mask: 0.6931  loss_rpn_cls: 0.09284  loss_rpn_loc: 0.02855  time: 1.3001  data_time: 0.6114  lr: 1.9981e-05  max_mem: 5155M
[06/30 04:08:59 d2.utils.events]:  eta: 0:11:09  iter: 39  total_loss: 3.071  loss_cls: 1.971  loss_box_reg: 0.1629  loss_mask: 0.6868  loss_rpn_cls: 0.1706  loss_rpn_loc: 0.04149  time: 1.2584  data_time: 0.4368  lr: 3.9961e-05  max_mem: 5155M
[06/30 04:09:26 d2.utils.events]:  eta: 0:11:01  iter: 59  total_loss: 2.057  loss_cls: 0.907  loss_box_reg: 0.1617  loss_mask: 0.6728  loss_rpn_cls: 0.04318  loss_rpn_loc: 0.02163  time: 1.2967  data_time: 0.5818  lr: 5.9941e-05  max_mem: 5155M
[06/30 04:09:53 d2.utils.events]:  eta: 0:10:30  iter: 79  total_loss: 1.275  loss_cls: 0.3573  loss_box_reg: 0.1221  loss_mask: 0.6509  loss_rpn_cls: 0.1325  loss_rpn_loc: 0.03614  time: 1

In [ ]:
# Métricas obtenidas del modelo usando el utilitario tensorboard
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
# Configuración del modelo para realizar predicciones sobre los datos de prueba
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # ruta del modelo ya entrenado
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # configuración del límite de detección
predictor = DefaultPredictor(cfg)

time: 994 ms (started: 2021-06-30 04:21:08 +00:00)


In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_TACO_filenames("data")
for d in random.sample(dataset_dicts, 1):    
    im = cv2.imread(d)
    outputs = predictor(im)  
    v = Visualizer(im[:, :, ::-1],
                   metadata=TACO_10_metadata, 
                   scale=0.5,  
                   instance_mode=ColorMode.IMAGE_BW   # remueve los colores de los pixeles no segmentados. Solo disponible para modelos de segmentados
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("TACO_10_test", ("bbox","segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "TACO_10_test")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

[06/30 04:21:11 d2.evaluation.coco_evaluation]: 'TACO_10_test' is not registered by `register_coco_instances`. Therefore trying to convert it to COCO format ...
[06/30 04:21:11 d2.data.datasets.coco]: Converting annotations of dataset 'TACO_10_test' to COCO format ...)
[06/30 04:21:11 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[06/30 04:21:11 d2.data.datasets.coco]: Conversion finished, #images: 300, #annotations: 927
[06/30 04:21:11 d2.data.datasets.coco]: Caching COCO format annotations at './output/TACO_10_test_coco_format.json' ...
[06/30 04:21:12 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|      Can      | 59           |   Other    | 346          |   Bottle   | 75           |
|  Bottle cap   | 49           |    Cup     | 47           |    Lid     | 19         

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[06/30 04:21:44 d2.evaluation.evaluator]: Inference done 11/300. 0.1549 s / img. ETA=0:14:32
[06/30 04:21:49 d2.evaluation.evaluator]: Inference done 14/300. 0.1421 s / img. ETA=0:12:41
[06/30 04:21:55 d2.evaluation.evaluator]: Inference done 16/300. 0.1387 s / img. ETA=0:12:38
[06/30 04:22:00 d2.evaluation.evaluator]: Inference done 20/300. 0.1311 s / img. ETA=0:10:49
[06/30 04:22:06 d2.evaluation.evaluator]: Inference done 24/300. 0.1266 s / img. ETA=0:09:53
[06/30 04:22:14 d2.evaluation.evaluator]: Inference done 27/300. 0.1277 s / img. ETA=0:10:00
[06/30 04:22:21 d2.evaluation.evaluator]: Inference done 30/300. 0.1279 s / img. ETA=0:09:59
[06/30 04:22:27 d2.evaluation.evaluator]: Inference done 33/300. 0.1277 s / img. ETA=0:09:46
[06/30 04:22:37 d2.evaluation.evaluator]: Inference done 34/300. 0.1350 s / img. ETA=0:10:58
[06/30 04:22:46 d2.evaluation.evaluator]: Inference done 37/300. 0.1366 s / img. ETA=0:11:03
[06/30 04:22:53 d2.evaluation.evaluator]: Inference done 39/300. 0.137